<a href="https://colab.research.google.com/github/HoseinNekouei/Emotions_Kaggle_CNN/blob/main/FER2013_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!nvidia-smi

Fri Jun  2 17:38:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
# Install the Kaggle Library
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Make a directory namde ".Kaggle"
! rm -r ~/.kaggle
! mkdir ~/.kaggle

In [7]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hoseinnekouei","key":"34e4ab84293ea07bb056bb90dbb9733c"}'}

In [8]:
# copy the "Kaggle.json" into the new directory
! mv kaggle.json ~/.kaggle/

In [9]:
# Allocate the required permission for this file.
! chmod 600 ~/.kaggle/kaggle.json

In [18]:
# download Dataset
! kaggle datasets download -d aspiring1/fer2013-images

 99% 121M/122M [00:03<00:00, 42.6MB/s]
100% 122M/122M [00:03<00:00, 35.0MB/s]


 move zip file from mydrive to content

! cp /content/drive/MyDrive/fer2013-images.zip /content/datasets/fer2013-images.zip

In [19]:
# unzip dataset
! rm -r Datasets/
! mkdir Datasets

! unzip -q /content/fer2013-images.zip -d /content/Datasets
! rm /content/fer2013-images.zip


In [20]:
! rm -r /content/Datasets/images

In [21]:
from glob import glob
import cv2 as cv
import numpy as np
from sklearn.model_selection import train_test_split


In [35]:
datalist = []
labels = []

for index, item in enumerate(glob('/content/Datasets/images_fer2013/Training/*/*')):
  img = cv.imread(item)
  img = cv.resize(img,(48,48))
  img = img / 255.0

  datalist.append(img)

  label = item.split('/')[-2]
  labels.append(label)

  if index % 1000 == 0:
    print(f'[Info]: {index}/28700 images processed !')

datalist = np.array(datalist)

x_train, x_test, y_train, y_test = train_test_split(
        datalist, labels, test_size=0.20, random_state=42)

[Info]: 0/28700 images processed !
[Info]: 1000/28700 images processed !
[Info]: 2000/28700 images processed !
[Info]: 3000/28700 images processed !
[Info]: 4000/28700 images processed !
[Info]: 5000/28700 images processed !
[Info]: 6000/28700 images processed !
[Info]: 7000/28700 images processed !
[Info]: 8000/28700 images processed !
[Info]: 9000/28700 images processed !
[Info]: 10000/28700 images processed !
[Info]: 11000/28700 images processed !
[Info]: 12000/28700 images processed !
[Info]: 13000/28700 images processed !
[Info]: 14000/28700 images processed !
[Info]: 15000/28700 images processed !
[Info]: 16000/28700 images processed !
[Info]: 17000/28700 images processed !
[Info]: 18000/28700 images processed !
[Info]: 19000/28700 images processed !
[Info]: 20000/28700 images processed !
[Info]: 21000/28700 images processed !
[Info]: 22000/28700 images processed !
[Info]: 23000/28700 images processed !
[Info]: 24000/28700 images processed !
[Info]: 25000/28700 images processed !

In [ ]:
! pip install keras

In [42]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [51]:
from keras import models, layers
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [61]:
from keras.layers.core.dense import Dense
opt = Adam(learning_rate = 0.001, decay = 0.00025)

aug = ImageDataGenerator(rotation_range = 20,
                         horizontal_flip=True,
                         zoom_range = 0.1,
                         brightness_range =(0.4,1.5),
                         fill_mode='nearest')

net = models.Sequential([
                         layers.Conv2D(32,(3,3), strides=(2,2),padding='same', input_shape=(48,48,3)),
                         layers.LeakyReLU(alpha=0.1),
                         layers.BatchNormalization(),
                         layers.Dropout(0.4),

                         layers.Conv2D(64,(3,3), strides=(2,2), padding='same'),
                         layers.LeakyReLU(alpha=0.1),
                         layers.BatchNormalization(),
                         layers.Dropout(0.4),

                         layers.Conv2D(128,(3,3), strides=(2,2),padding='same'),
                         layers.LeakyReLU(alpha=0.1),
                         layers.BatchNormalization(),
                         layers.Dropout(0.4),

                         layers.Conv2D(128,(3,3), strides=(2,2),padding='same'),
                         layers.LeakyReLU(alpha=0.1),
                         layers.BatchNormalization(),
                         layers.Dropout(0.4),

                         layers.Flatten(),

                         layers.Dense(1024),
                         layers.LeakyReLU(alpha=0.1),
                         layers.BatchNormalization(),
                         layers.Dropout(0.4),
                        
                        layers.Dense(7, activation='softmax')
                         ])

net.compile(opt,
            loss='categorical_crossentropy',
            metrics = ['accuracy'])

net.summary()



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 24, 24, 32)        896       
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 24, 24, 32)        0         
                                                                 
 batch_normalization_10 (Bat  (None, 24, 24, 32)       128       
 chNormalization)                                                
                                                                 
 dropout_10 (Dropout)        (None, 24, 24, 32)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 12, 12, 64)        18496     
                                                                 
 leaky_re_lu_11 (LeakyReLU)  (None, 12, 12, 64)        0         
                                                      

In [ ]:
BATCHSIZE = 32
EPOCHS = 50
    
H = net.fit(aug.flow(x_train, y_train, batch_size=BATCHSIZE),
            steps_per_epoch=len(x_train)//BATCHSIZE,
            epochs=EPOCHS, validation_data=(x_test, y_test))

loss, accuracy = net.evaluate(x_test, y_test)

print(f'Test loss: {loss :.2f}, Test accuracy: {accuracy :.2f}')

Epoch 1/50
717/717 [==============================] - 44s 47ms/step - loss: 2.3852 - accuracy: 0.1982 - val_loss: 25.4193 - val_accuracy: 0.1728
Epoch 2/50
717/717 [==============================] - 53s 74ms/step - loss: 2.0165 - accuracy: 0.2225 - val_loss: 24.8277 - val_accuracy: 0.1388
Epoch 3/50
717/717 [==============================] - 39s 55ms/step - loss: 1.9472 - accuracy: 0.2275 - val_loss: 39.2822 - val_accuracy: 0.1743
Epoch 4/50
717/717 [==============================] - 30s 42ms/step - loss: 1.9071 - accuracy: 0.2343 - val_loss: 48.7096 - val_accuracy: 0.1465
Epoch 5/50
717/717 [==============================] - 30s 42ms/step - loss: 1.8969 - accuracy: 0.2342 - val_loss: 27.5516 - val_accuracy: 0.1506
Epoch 6/50
717/717 [==============================] - 30s 42ms/step - loss: 1.8879 - accuracy: 0.2395 - val_loss: 19.6264 - val_accuracy: 0.1073
Epoch 7/50
717/717 [==============================] - 30s 42ms/step - loss: 1.8760 - accuracy: 0.2398 - val_loss: 17.6660 - val_ac

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.plot(np.arange(EPOCHS), H.history['loss'], label='Train loss')
plt.plot(np.arange(EPOCHS), H.history['accuracy'], label='Train accuracy')
plt.plot(np.arange(EPOCHS), H.history['val_loss'], label='Test loss')
plt.plot(np.arange(EPOCHS), H.history['val_accuracy'], label='Test accuracy')

plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss/Accuracy')
plt.title('emotions dataset Training')

plt.show()